# Supply Chain Emissions Modeling Using Industry and Commodity Data (2010–2016)

**Problem Statement:**

You have annual supply chain emission data from 2010–2016 categorized into industries and commodities. The goal is to develop a regression model that can predict the Supply Chain Emission Factors with Margins based on descriptive and quality metrics (substance, unit, reliability, temporal/geographical/technological/data collection correlations, etc.).

# 🌱 Greenhouse Gas Emission Prediction Project

**Project Goal:**  
To analyze and predict greenhouse gas (GHG) emissions from various U.S. industries and commodities using the official dataset from [data.gov](https://catalog.data.gov/dataset/supply-chain-greenhouse-gas-emission-factors-for-us-industries-and-commodities).

**Source:**  
[Supply Chain Greenhouse Gas Emission Factors](https://catalog.data.gov/dataset/supply-chain-greenhouse-gas-emission-factors-for-us-industries-and-commodities)

**Tools:** Python, Pandas, Scikit-learn, Matplotlib, Seaborn

## 📂 Dataset Overview

This dataset contains supply chain emission factors associated with various U.S. industries and commodities.

**Key Columns:**
- `Code`: Industry/Commodity classification code
- `Name`: Name of the industry/commodity
- `Substance`: Type of greenhouse gas (CO2, methane, nitrous oxide, etc.)
- `Unit`: Measurement units (e.g., kg/2018 USD, purchaser price)
- `Supply Chain Emission Factors with Margins`: Target variable for prediction
- `DQ ReliabilityScore`: Data quality reliability score
- `DQ TemporalCorrelation`: Temporal correlation quality score
- `DQ GeographicalCorrelation`: Geographical correlation quality score
- `DQ TechnologicalCorrelation`: Technological correlation quality score
- `DQ DataCollection`: Data collection quality score

## 🧹 Data Preprocessing

Steps:
- Handle missing values
- Convert units where needed
- Encode categorical features
- Normalize/scale numeric columns

## 🤖 Model Building & Evaluation

We aim to predict `Supply Chain Emission Factors with Margins` using regression models.

Models to try:
- Linear Regression
- Random Forest Regressor

**Evaluation Metrics:**
- RMSE (Root Mean Squared Error)
- MAE (Mean Absolute Error)
- R² Score

##### Steps:
- Step 1: Import Required Libraries
- Step 2: Load Dataset
- Step 3: Data Preprocessing (EDA+Cleaning+Encoding)
- Step 4: Training
- Step 5: Prediction and Evaluation
- Step 6: Hyperparameter Tuning
- Step 7: Comparative Study and Selecting the Best model

# Step 1: Import Required Libraries

In [ ]:
# Core libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

# Machine learning libraries
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Model persistence
import joblib
import pickle

# File handling
import os
from pathlib import Path

print("Libraries imported successfully!")

# Step 2: Load Dataset

In [ ]:
# Define the Excel file path and years range
excel_file = 'SupplyChainEmissionFactorsforUSIndustriesCommodities.xlsx'
years = range(2010, 2017)

print(f"Years to process: {list(years)}")
print(f"Looking for file: {excel_file}")

In [ ]:
# Check available sheets in the Excel file
if os.path.exists(excel_file):
    xl_file = pd.ExcelFile(excel_file)
    available_sheets = xl_file.sheet_names
    print(f"Available sheets in Excel file: {available_sheets[:10]}...")  # Show first 10 sheets
    print(f"Total sheets: {len(available_sheets)}")
else:
    print(f"Error: File '{excel_file}' not found!")

In [ ]:
# Load first year data to understand structure
df_commodity_sample = pd.read_excel(excel_file, sheet_name=f'{years[0]}_Detail_Commodity')
print(f"Commodity data shape for {years[0]}: {df_commodity_sample.shape}")
print(f"Columns: {list(df_commodity_sample.columns)}")
df_commodity_sample.head()

In [ ]:
# Load industry data to understand structure
df_industry_sample = pd.read_excel(excel_file, sheet_name=f'{years[0]}_Detail_Industry')
print(f"Industry data shape for {years[0]}: {df_industry_sample.shape}")
print(f"Columns: {list(df_industry_sample.columns)}")
df_industry_sample.head()

In [ ]:
# Load and combine all data from all years
all_data = []

for year in years:
    try:
        # Load commodity data
        df_com = pd.read_excel(excel_file, sheet_name=f'{year}_Detail_Commodity')
        # Load industry data
        df_ind = pd.read_excel(excel_file, sheet_name=f'{year}_Detail_Industry')
        
        # Add source and year columns
        df_com['Source'] = 'Commodity'
        df_ind['Source'] = 'Industry'
        df_com['Year'] = year
        df_ind['Year'] = year
        
        # Clean column names
        df_com.columns = df_com.columns.str.strip()
        df_ind.columns = df_ind.columns.str.strip()

        # Standardize column names
        df_com.rename(columns={
            'Commodity Code': 'Code',
            'Commodity Name': 'Name'
        }, inplace=True)
        
        df_ind.rename(columns={
            'Industry Code': 'Code',
            'Industry Name': 'Name'
        }, inplace=True)
        
        # Combine commodity and industry data for this year
        year_data = pd.concat([df_com, df_ind], ignore_index=True)
        all_data.append(year_data)
        
        print(f"Loaded {len(year_data)} records for year {year}")
        
    except Exception as e:
        print(f"Error processing year {year}: {e}")

# Combine all years
if all_data:
    df = pd.concat(all_data, ignore_index=True)
    print(f"\nTotal combined dataset shape: {df.shape}")
    print(f"Years in dataset: {sorted(df['Year'].unique())}")
else:
    print("No data was loaded!")

In [ ]:
# Display basic information about the dataset
print("📊 Dataset Basic Information:")
print(f"Shape: {df.shape}")
print(f"\nColumns: {list(df.columns)}")
print(f"\nData types:")
print(df.dtypes)
print(f"\nFirst few rows:")
df.head()

# Step 3: Data Preprocessing (EDA + Cleaning + Encoding)

## 3.1 Exploratory Data Analysis (EDA)

In [ ]:
# Basic statistics
print("📈 Dataset Summary Statistics:")
print(f"Total records: {len(df):,}")
print(f"Number of features: {df.shape[1]}")
print(f"Years covered: {sorted(df['Year'].unique())}")
print(f"Unique codes: {df['Code'].nunique()}")
print(f"Unique substances: {df['Substance'].nunique()}")
print(f"Sources: {df['Source'].value_counts().to_dict()}")

# Missing values analysis
print("\n🔍 Missing Values Analysis:")
missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing_data,
    'Missing Percentage': missing_percent
})
missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False)
if len(missing_df) > 0:
    print(missing_df)
else:
    print("No missing values found!")

In [ ]:
# Target variable analysis
target_col = 'Supply Chain Emission Factors with Margins'

print(f"🎯 Target Variable Analysis: {target_col}")
print(f"Statistics:")
print(df[target_col].describe())

# Visualization of target variable
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Histogram
axes[0, 0].hist(df[target_col], bins=50, alpha=0.7, color='skyblue')
axes[0, 0].set_title('Distribution of Target Variable')
axes[0, 0].set_xlabel(target_col)
axes[0, 0].set_ylabel('Frequency')

# Box plot
axes[0, 1].boxplot(df[target_col])
axes[0, 1].set_title('Box Plot of Target Variable')
axes[0, 1].set_ylabel(target_col)

# Log-scale histogram (for better visualization if data is skewed)
axes[1, 0].hist(np.log1p(df[target_col]), bins=50, alpha=0.7, color='lightgreen')
axes[1, 0].set_title('Log-scale Distribution of Target Variable')
axes[1, 0].set_xlabel(f'log1p({target_col})')
axes[1, 0].set_ylabel('Frequency')

# Target by year
yearly_stats = df.groupby('Year')[target_col].agg(['mean', 'median', 'std']).reset_index()
axes[1, 1].plot(yearly_stats['Year'], yearly_stats['mean'], marker='o', label='Mean', linewidth=2)
axes[1, 1].plot(yearly_stats['Year'], yearly_stats['median'], marker='s', label='Median', linewidth=2)
axes[1, 1].set_title('Target Variable Trends by Year')
axes[1, 1].set_xlabel('Year')
axes[1, 1].set_ylabel(target_col)
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Categorical variables analysis
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Substance distribution
substance_counts = df['Substance'].value_counts()
axes[0, 0].pie(substance_counts.values, labels=substance_counts.index, autopct='%1.1f%%')
axes[0, 0].set_title('Distribution of Substances')

# Unit distribution
unit_counts = df['Unit'].value_counts()
axes[0, 1].pie(unit_counts.values[:5], labels=unit_counts.index[:5], autopct='%1.1f%%')  # Top 5 units
axes[0, 1].set_title('Distribution of Units (Top 5)')

# Target by Substance
substance_target = df.groupby('Substance')[target_col].mean().sort_values(ascending=True)
axes[1, 0].barh(range(len(substance_target)), substance_target.values, color='lightcoral')
axes[1, 0].set_yticks(range(len(substance_target)))
axes[1, 0].set_yticklabels(substance_target.index)
axes[1, 0].set_title('Average Target Value by Substance')
axes[1, 0].set_xlabel('Average Emission Factor')

# Target by Source
source_target = df.groupby('Source')[target_col].mean()
axes[1, 1].bar(source_target.index, source_target.values, color=['lightblue', 'lightgreen'])
axes[1, 1].set_title('Average Target Value by Source')
axes[1, 1].set_ylabel('Average Emission Factor')

plt.tight_layout()
plt.show()

In [ ]:
# Data Quality scores analysis
quality_cols = [
    'DQ ReliabilityScore of Factors without Margins',
    'DQ TemporalCorrelation of Factors without Margins',
    'DQ GeographicalCorrelation of Factors without Margins',
    'DQ TechnologicalCorrelation of Factors without Margins',
    'DQ DataCollection of Factors without Margins'
]

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for i, col in enumerate(quality_cols):
    df[col].value_counts().sort_index().plot(kind='bar', ax=axes[i], color='skyblue')
    axes[i].set_title(f'Distribution of {col.split(" ")[1]}')
    axes[i].set_xlabel('Score')
    axes[i].set_ylabel('Count')
    axes[i].tick_params(axis='x', rotation=0)

# Correlation between quality scores and target variable
quality_correlations = df[quality_cols + [target_col]].corr()[target_col].drop(target_col)
axes[5].barh(range(len(quality_correlations)), quality_correlations.values, color='lightcoral')
axes[5].set_yticks(range(len(quality_correlations)))
axes[5].set_yticklabels([col.split(' ')[1] for col in quality_correlations.index])
axes[5].set_title('Correlation with Target Variable')
axes[5].set_xlabel('Correlation Coefficient')

plt.tight_layout()
plt.show()

## 3.2 Data Cleaning

In [ ]:
# Data cleaning steps
print("🧹 Data Cleaning Steps:")
print(f"Original dataset shape: {df.shape}")

# Remove rows with missing target values
df_clean = df.dropna(subset=[target_col]).copy()
print(f"After removing missing target values: {df_clean.shape}")

# Drop unnecessary columns
columns_to_drop = ['Unnamed: 7'] if 'Unnamed: 7' in df_clean.columns else []
if columns_to_drop:
    df_clean = df_clean.drop(columns=columns_to_drop)
    print(f"After dropping unnecessary columns: {df_clean.shape}")

# Handle any remaining missing values in features
missing_before = df_clean.isnull().sum().sum()
if missing_before > 0:
    print(f"Missing values before cleaning: {missing_before}")
    # Fill missing values with appropriate methods
    for col in df_clean.columns:
        if df_clean[col].dtype in ['float64', 'int64']:
            df_clean[col].fillna(df_clean[col].median(), inplace=True)
        else:
            df_clean[col].fillna(df_clean[col].mode()[0] if not df_clean[col].mode().empty else 'Unknown', inplace=True)
    
    missing_after = df_clean.isnull().sum().sum()
    print(f"Missing values after cleaning: {missing_after}")

print(f"\nFinal cleaned dataset shape: {df_clean.shape}")

## 3.3 Feature Engineering

In [ ]:
# Feature engineering
print("⚙️ Feature Engineering:")

# Create composite data quality score
quality_cols = [
    'DQ ReliabilityScore of Factors without Margins',
    'DQ TemporalCorrelation of Factors without Margins',
    'DQ GeographicalCorrelation of Factors without Margins',
    'DQ TechnologicalCorrelation of Factors without Margins',
    'DQ DataCollection of Factors without Margins'
]

df_clean['DQ_Composite_Score'] = df_clean[quality_cols].mean(axis=1)
print(f"Created composite data quality score")

# Create margin ratio feature
df_clean['Margin_Ratio'] = (df_clean['Margins of Supply Chain Emission Factors'] / 
                           df_clean['Supply Chain Emission Factors without Margins'])
df_clean['Margin_Ratio'] = df_clean['Margin_Ratio'].replace([np.inf, -np.inf], 0).fillna(0)
print(f"Created margin ratio feature")

# Create categorical features for code prefixes
df_clean['Code_Prefix'] = df_clean['Code'].astype(str).str[:4]
print(f"Created code prefix feature")

# Create decade feature
df_clean['Decade'] = (df_clean['Year'] // 10) * 10
print(f"Created decade feature")

print(f"\nDataset shape after feature engineering: {df_clean.shape}")
print(f"New features created: DQ_Composite_Score, Margin_Ratio, Code_Prefix, Decade")

## 3.4 Feature Encoding and Preprocessing

In [ ]:
# Select features for modeling
print("🎯 Feature Selection for Modeling:")

# Define feature categories
categorical_features = ['Substance', 'Unit', 'Source', 'Code_Prefix']
numerical_features = [
    'Supply Chain Emission Factors without Margins',
    'Margins of Supply Chain Emission Factors',
    'DQ ReliabilityScore of Factors without Margins',
    'DQ TemporalCorrelation of Factors without Margins',
    'DQ GeographicalCorrelation of Factors without Margins',
    'DQ TechnologicalCorrelation of Factors without Margins',
    'DQ DataCollection of Factors without Margins',
    'DQ_Composite_Score',
    'Margin_Ratio',
    'Year'
]

# Create feature matrix
X = df_clean[categorical_features + numerical_features].copy()
y = df_clean[target_col].copy()

print(f"Feature matrix shape: {X.shape}")
print(f"Target vector shape: {y.shape}")
print(f"Categorical features: {categorical_features}")
print(f"Numerical features: {numerical_features}")

In [ ]:
# Encode categorical variables
print("🔄 Encoding Categorical Variables:")

# Label encoding for categorical variables with many categories
label_encoders = {}
X_encoded = X.copy()

for col in categorical_features:
    le = LabelEncoder()
    X_encoded[col] = le.fit_transform(X_encoded[col].astype(str))
    label_encoders[col] = le
    print(f"Label encoded {col}: {len(le.classes_)} unique values")

print(f"\nEncoded feature matrix shape: {X_encoded.shape}")
print(f"All features are now numerical: {X_encoded.dtypes.value_counts().to_dict()}")

# Step 4: Model Training

In [ ]:
# Split the data into training and testing sets
print("📊 Splitting Data:")

X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42, stratify=df_clean['Source']
)

print(f"Training set shape: {X_train.shape}, {y_train.shape}")
print(f"Testing set shape: {X_test.shape}, {y_test.shape}")
print(f"Training set target statistics:")
print(y_train.describe())
print(f"\nTesting set target statistics:")
print(y_test.describe())

In [ ]:
# Feature scaling
print("📏 Feature Scaling:")

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Scaled training features shape: {X_train_scaled.shape}")
print(f"Scaled testing features shape: {X_test_scaled.shape}")
print(f"Feature scaling completed using StandardScaler")

In [ ]:
# Train baseline models
print("🤖 Training Baseline Models:")

# Define models
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42)
}

# Train and evaluate models
model_results = {}

for name, model in models.items():
    print(f"\nTraining {name}...")
    
    # For Linear Regression, use scaled features
    if name == 'Linear Regression':
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
    else:
        # For tree-based models, scaling is not necessary
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
    
    # Calculate metrics
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    model_results[name] = {
        'model': model,
        'predictions': y_pred,
        'rmse': rmse,
        'mae': mae,
        'r2': r2
    }
    
    print(f"  RMSE: {rmse:.4f}")
    print(f"  MAE: {mae:.4f}")
    print(f"  R² Score: {r2:.4f}")

print("\n✅ Baseline model training completed!")

# Step 5: Model Evaluation and Visualization

In [ ]:
# Compare model performance
print("📊 Model Performance Comparison:")

# Create comparison DataFrame
comparison_df = pd.DataFrame({
    'Model': list(model_results.keys()),
    'RMSE': [model_results[name]['rmse'] for name in model_results.keys()],
    'MAE': [model_results[name]['mae'] for name in model_results.keys()],
    'R² Score': [model_results[name]['r2'] for name in model_results.keys()]
})

print(comparison_df.to_string(index=False))

# Visualize model performance
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# RMSE comparison
axes[0].bar(comparison_df['Model'], comparison_df['RMSE'], color='lightcoral')
axes[0].set_title('RMSE Comparison')
axes[0].set_ylabel('RMSE')
axes[0].tick_params(axis='x', rotation=45)

# MAE comparison
axes[1].bar(comparison_df['Model'], comparison_df['MAE'], color='lightblue')
axes[1].set_title('MAE Comparison')
axes[1].set_ylabel('MAE')
axes[1].tick_params(axis='x', rotation=45)

# R² Score comparison
axes[2].bar(comparison_df['Model'], comparison_df['R² Score'], color='lightgreen')
axes[2].set_title('R² Score Comparison')
axes[2].set_ylabel('R² Score')
axes[2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Actual vs Predicted plots
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

for i, (name, results) in enumerate(model_results.items()):
    axes[i].scatter(y_test, results['predictions'], alpha=0.6)
    axes[i].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
    axes[i].set_xlabel('Actual Values')
    axes[i].set_ylabel('Predicted Values')
    axes[i].set_title(f'{name}: Actual vs Predicted\nR² = {results["r2"]:.4f}')
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Residual analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.flatten()

for i, (name, results) in enumerate(model_results.items()):
    residuals = y_test - results['predictions']
    
    # Residuals vs Predicted
    axes[i*2].scatter(results['predictions'], residuals, alpha=0.6)
    axes[i*2].axhline(y=0, color='r', linestyle='--')
    axes[i*2].set_xlabel('Predicted Values')
    axes[i*2].set_ylabel('Residuals')
    axes[i*2].set_title(f'{name}: Residuals vs Predicted')
    axes[i*2].grid(True, alpha=0.3)
    
    # Residuals histogram
    axes[i*2+1].hist(residuals, bins=30, alpha=0.7, color='skyblue')
    axes[i*2+1].set_xlabel('Residuals')
    axes[i*2+1].set_ylabel('Frequency')
    axes[i*2+1].set_title(f'{name}: Residuals Distribution')
    axes[i*2+1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Feature importance analysis (for Random Forest)
rf_model = model_results['Random Forest']['model']
feature_names = categorical_features + numerical_features

# Get feature importances
importances = rf_model.feature_importances_
feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values('Importance', ascending=False)

print("🎯 Feature Importance (Random Forest):")
print(feature_importance_df.to_string(index=False))

# Plot feature importance
plt.figure(figsize=(12, 8))
plt.barh(range(len(feature_importance_df)), feature_importance_df['Importance'], color='lightblue')
plt.yticks(range(len(feature_importance_df)), feature_importance_df['Feature'])
plt.xlabel('Feature Importance')
plt.title('Random Forest Feature Importance')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

# Step 6: Hyperparameter Tuning

In [ ]:
# Hyperparameter tuning for Random Forest
print("🔧 Hyperparameter Tuning for Random Forest:")

# Define parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Create Random Forest model
rf = RandomForestRegressor(random_state=42)

# Perform grid search
print("Performing Grid Search (this may take a few minutes)...")
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=5,
    scoring='r2',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train, y_train)

print(f"\n✅ Grid Search completed!")
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation score: {grid_search.best_score_:.4f}")

In [ ]:
# Evaluate tuned model
print("📊 Evaluating Tuned Random Forest Model:")

# Get the best model
best_rf = grid_search.best_estimator_

# Make predictions
y_pred_tuned = best_rf.predict(X_test)

# Calculate metrics
rmse_tuned = np.sqrt(mean_squared_error(y_test, y_pred_tuned))
mae_tuned = mean_absolute_error(y_test, y_pred_tuned)
r2_tuned = r2_score(y_test, y_pred_tuned)

print(f"Tuned Random Forest Performance:")
print(f"  RMSE: {rmse_tuned:.4f}")
print(f"  MAE: {mae_tuned:.4f}")
print(f"  R² Score: {r2_tuned:.4f}")

# Compare with original Random Forest
original_rf_r2 = model_results['Random Forest']['r2']
improvement = r2_tuned - original_rf_r2
print(f"\nImprovement over original Random Forest:")
print(f"  R² Score improvement: {improvement:.4f}")
print(f"  Percentage improvement: {(improvement/original_rf_r2)*100:.2f}%")

# Add tuned model to results
model_results['Random Forest (Tuned)'] = {
    'model': best_rf,
    'predictions': y_pred_tuned,
    'rmse': rmse_tuned,
    'mae': mae_tuned,
    'r2': r2_tuned
}

In [ ]:
# Cross-validation analysis
print("🔄 Cross-Validation Analysis:")

# Perform cross-validation for all models
cv_results = {}

for name, results in model_results.items():
    model = results['model']
    
    if name == 'Linear Regression':
        cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='r2')
    else:
        cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='r2')
    
    cv_results[name] = {
        'mean_score': cv_scores.mean(),
        'std_score': cv_scores.std(),
        'scores': cv_scores
    }
    
    print(f"{name}:")
    print(f"  CV Mean R² Score: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")
    print(f"  CV Scores: {cv_scores}")
    print()

# Visualize cross-validation results
plt.figure(figsize=(12, 6))
model_names = list(cv_results.keys())
cv_means = [cv_results[name]['mean_score'] for name in model_names]
cv_stds = [cv_results[name]['std_score'] for name in model_names]

plt.bar(model_names, cv_means, yerr=cv_stds, capsize=5, color=['lightcoral', 'lightblue', 'lightgreen'])
plt.xlabel('Models')
plt.ylabel('Cross-Validation R² Score')
plt.title('Cross-Validation Performance Comparison')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Step 7: Comparative Study and Best Model Selection

In [ ]:
# Final model comparison
print("🏆 Final Model Comparison and Selection:")

# Create comprehensive comparison
final_comparison = pd.DataFrame({
    'Model': list(model_results.keys()),
    'Test_RMSE': [model_results[name]['rmse'] for name in model_results.keys()],
    'Test_MAE': [model_results[name]['mae'] for name in model_results.keys()],
    'Test_R2': [model_results[name]['r2'] for name in model_results.keys()],
    'CV_R2_Mean': [cv_results[name]['mean_score'] for name in model_results.keys()],
    'CV_R2_Std': [cv_results[name]['std_score'] for name in model_results.keys()]
})

# Calculate overfitting indicator (difference between CV and test performance)
final_comparison['Overfitting_Indicator'] = final_comparison['CV_R2_Mean'] - final_comparison['Test_R2']

print("\n📊 Comprehensive Model Comparison:")
print(final_comparison.round(4).to_string(index=False))

# Select best model based on multiple criteria
best_model_idx = final_comparison['Test_R2'].idxmax()
best_model_name = final_comparison.loc[best_model_idx, 'Model']
best_model = model_results[best_model_name]['model']

print(f"\n🎯 Best Model Selection:")
print(f"Selected Model: {best_model_name}")
print(f"Selection Criteria: Highest Test R² Score")
print(f"\nBest Model Performance:")
print(f"  Test RMSE: {final_comparison.loc[best_model_idx, 'Test_RMSE']:.4f}")
print(f"  Test MAE: {final_comparison.loc[best_model_idx, 'Test_MAE']:.4f}")
print(f"  Test R² Score: {final_comparison.loc[best_model_idx, 'Test_R2']:.4f}")
print(f"  CV R² Score: {final_comparison.loc[best_model_idx, 'CV_R2_Mean']:.4f} (+/- {final_comparison.loc[best_model_idx, 'CV_R2_Std']:.4f})")
print(f"  Overfitting Indicator: {final_comparison.loc[best_model_idx, 'Overfitting_Indicator']:.4f}")

In [ ]:
# Model interpretation and insights
print("🔍 Model Interpretation and Insights:")

if 'Random Forest' in best_model_name:
    # Feature importance for Random Forest
    feature_names = categorical_features + numerical_features
    importances = best_model.feature_importances_
    
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': importances
    }).sort_values('Importance', ascending=False)
    
    print("\n🎯 Top 10 Most Important Features:")
    print(importance_df.head(10).to_string(index=False))
    
    # Plot top features
    plt.figure(figsize=(10, 6))
    top_features = importance_df.head(10)
    plt.barh(range(len(top_features)), top_features['Importance'], color='lightblue')
    plt.yticks(range(len(top_features)), top_features['Feature'])
    plt.xlabel('Feature Importance')
    plt.title(f'Top 10 Feature Importance - {best_model_name}')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

elif 'Linear Regression' in best_model_name:
    # Coefficients for Linear Regression
    feature_names = categorical_features + numerical_features
    coefficients = best_model.coef_
    
    coef_df = pd.DataFrame({
        'Feature': feature_names,
        'Coefficient': coefficients
    }).sort_values('Coefficient', key=abs, ascending=False)
    
    print("\n📈 Top 10 Most Influential Features (by coefficient magnitude):")
    print(coef_df.head(10).to_string(index=False))
    
    # Plot top coefficients
    plt.figure(figsize=(10, 6))
    top_coefs = coef_df.head(10)
    colors = ['red' if x < 0 else 'blue' for x in top_coefs['Coefficient']]
    plt.barh(range(len(top_coefs)), top_coefs['Coefficient'], color=colors)
    plt.yticks(range(len(top_coefs)), top_coefs['Feature'])
    plt.xlabel('Coefficient Value')
    plt.title(f'Top 10 Feature Coefficients - {best_model_name}')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

In [ ]:
# Generate insights and conclusions
print("💡 Key Insights and Conclusions:")
print("\n1. Data Quality Impact:")
quality_corr = df_clean[quality_cols + [target_col]].corr()[target_col].drop(target_col)
print(f"   - Data quality scores show varying correlations with emission factors")
print(f"   - Highest correlation: {quality_corr.abs().idxmax()} ({quality_corr.abs().max():.3f})")

print("\n2. Substance Type Analysis:")
substance_stats = df_clean.groupby('Substance')[target_col].agg(['mean', 'std', 'count'])
print(f"   - {substance_stats['mean'].idxmax()} has the highest average emission factor ({substance_stats['mean'].max():.3f})")
print(f"   - {substance_stats['mean'].idxmin()} has the lowest average emission factor ({substance_stats['mean'].min():.3f})")

print("\n3. Source Comparison:")
source_stats = df_clean.groupby('Source')[target_col].agg(['mean', 'std', 'count'])
print(f"   - Industry vs Commodity emission factors:")
for source in source_stats.index:
    print(f"     {source}: Mean = {source_stats.loc[source, 'mean']:.3f}, Count = {source_stats.loc[source, 'count']}")

print("\n4. Model Performance Summary:")
print(f"   - Best performing model: {best_model_name}")
print(f"   - Achieved R² score of {final_comparison.loc[best_model_idx, 'Test_R2']:.3f} on test data")
print(f"   - Model explains {final_comparison.loc[best_model_idx, 'Test_R2']*100:.1f}% of variance in emission factors")

print("\n5. Recommendations:")
print("   - Focus on improving data quality scores for better predictions")
print("   - Consider substance-specific models for more accurate predictions")
print("   - Monitor model performance over time as new data becomes available")
print("   - Use the model for preliminary emission factor estimation and validation")

In [ ]:
# Save the best model and associated components
print("💾 Saving Best Model and Components:")

# Create timestamp for file naming
from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Save the best model
model_filename = f'best_ghg_emissions_model_{timestamp}.joblib'
joblib.dump(best_model, model_filename)
print(f"✅ Best model saved as: {model_filename}")

# Save model using pickle as backup
model_pickle_filename = f'best_ghg_emissions_model_{timestamp}.pkl'
with open(model_pickle_filename, 'wb') as f:
    pickle.dump(best_model, f)
print(f"✅ Model backup saved as: {model_pickle_filename}")

# Save scaler if Linear Regression was the best model
if 'Linear Regression' in best_model_name:
    scaler_filename = f'ghg_emissions_scaler_{timestamp}.joblib'
    joblib.dump(scaler, scaler_filename)
    print(f"✅ Scaler saved as: {scaler_filename}")

# Save label encoders
encoders_filename = f'ghg_emissions_encoders_{timestamp}.joblib'
joblib.dump(label_encoders, encoders_filename)
print(f"✅ Label encoders saved as: {encoders_filename}")

# Save model metadata
metadata = {
    'best_model_name': best_model_name,
    'feature_names': feature_names,
    'categorical_features': categorical_features,
    'numerical_features': numerical_features,
    'target_column': target_col,
    'test_performance': {
        'rmse': final_comparison.loc[best_model_idx, 'Test_RMSE'],
        'mae': final_comparison.loc[best_model_idx, 'Test_MAE'],
        'r2': final_comparison.loc[best_model_idx, 'Test_R2']
    },
    'cv_performance': {
        'mean_r2': final_comparison.loc[best_model_idx, 'CV_R2_Mean'],
        'std_r2': final_comparison.loc[best_model_idx, 'CV_R2_Std']
    },
    'training_date': timestamp,
    'dataset_shape': df_clean.shape
}

metadata_filename = f'ghg_emissions_model_metadata_{timestamp}.pkl'
with open(metadata_filename, 'wb') as f:
    pickle.dump(metadata, f)
print(f"✅ Model metadata saved as: {metadata_filename}")

print(f"\n🎉 Model training and evaluation completed successfully!")
print(f"📁 Files saved:")
print(f"   - {model_filename}")
print(f"   - {model_pickle_filename}")
print(f"   - {encoders_filename}")
print(f"   - {metadata_filename}")
if 'Linear Regression' in best_model_name:
    print(f"   - {scaler_filename}")

# 🎯 Project Summary

This notebook has successfully completed a comprehensive machine learning analysis for predicting greenhouse gas emission factors from U.S. supply chain data (2010-2016).

## Key Achievements:

1. **Data Processing**: Successfully loaded and combined multi-year Excel data from both commodity and industry sources
2. **Feature Engineering**: Created composite data quality scores, margin ratios, and categorical encodings
3. **Model Development**: Trained and compared Linear Regression and Random Forest models
4. **Hyperparameter Optimization**: Performed grid search to optimize Random Forest performance
5. **Model Evaluation**: Comprehensive evaluation using multiple metrics and cross-validation
6. **Model Selection**: Selected the best performing model based on test R² score
7. **Model Persistence**: Saved the best model and all associated components for future use

## Next Steps:

- Deploy the model for emission factor predictions
- Monitor model performance with new data
- Consider ensemble methods for improved accuracy
- Explore deep learning approaches for complex patterns

The trained model can now be used to predict supply chain emission factors based on substance type, data quality metrics, and other relevant features.